# Práctica 2 - Ejercicio 11

El dataset website tiene información sobre el tiempo de permanencia de sus usuarios en cada una de las páginas del sitio. El formato de los datos del dataset es:

`<id_user, id_page, time>`

Modele una solución MapReduce, sugiriendo la utilización de funciones combiners en los casos que considere necesario, que permita obtener:
* La página más visitada (la página en la que más tiempo permaneció) para cada
usuario
* El usuario que más páginas distintas visitó
* La página más visitada (en cuanto a cantidad de visitas, sin importar el tiempo de permanencia) por todos los usuarios.

Indique como queda el DAG del proceso completo (las tres consultas)


In [2]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Wed Oct 21 13:25:40 2020

@author: pablo
"""

import os

from MRE import Job

inputDir = "datasets/website/"
outputDir = "output/website"

> A medida que se ejecuta el script, se crea una carpeta para cada Job donde se almacenan los resultados parciales y totales.
### a. La página más visitada (la página en la que más tiempo permaneció) para cada usuario

El JobA1 calcula para cada usuario, el tiempo total de permanencia para cada una de las páginas que visitó. Para el combiner utilizo la misma función reduce

In [3]:
# <id_user, id_page, time>
def fmapA1(key, value, context):
    values = value.split()
    context.write((key, values[0]), values[1])

def fredA1(key, values, context):
    total = 0
    for v in values:
        total += int(v)
    context.write(key, total)

#Creo directorios a/1/ en directorio de output para los resultados del JobA1
os.makedirs(outputDir + '/a/1', exist_ok=True)

jobA1 = Job(inputDir, outputDir + '/a/1', fmapA1, fredA1)
jobA1.setCombiner(fredA1)
success = jobA1.waitForCompletion()

El JobA2 recibe para cada usuario, las páginas que visito con el tiempo total de permanencia en cada una y retorna para cada usuario la página en la que más tiempo permaneció

In [4]:
# <id_user, id_page, time>
def fmapA2(key, value, context):
    values = value.split()
    context.write(key, values)

# intenté ordernar la entrada del reduce fredA2 por el tiempo de permanencia, para evitar iterar sobre todas las páginas visitadas por el usuario .
# Como el orden se establece por la clave intermedia, no me fue posible lograrlo, al menos en esta solución.
def fredA2(key, values, context):
    max= (0,0)
    for v in values:
        if (int(max[1]) < int(v[1]) ):
            max = v
    context.write(key, max)

os.makedirs(outputDir + '/a/2', exist_ok=True)
jobA2 = Job(outputDir + '/a/1', outputDir + '/a/2', fmapA2, fredA2)
success = jobA2.waitForCompletion()

print(f"Ver resultados en '{outputDir}/a/2/output.txt'")

Ver resultados en 'output/website/a/2/output.txt'


![img](https://docs.google.com/drawings/d/e/2PACX-1vTR2vYsUGk_jLatYMwmCf4H6cxKGDSxnIgCLwA9OwWsPjr1-UmXU5c79A6cGZKhR646CwV_D22x26WV/pub?w=525&h=166)

### b. El usuario que más páginas distintas visitó

JobB1 calcula la canitdad de paginas visitadas por usuario. Para la función combiner utilizo la misma función usada en el reduce

In [5]:
# <id_user, id_page, time>
def fmapB1(key, value, context):
    values = value.split()
    context.write((key,values[0]), 1)

def fredB1(key, values, context):
    total = 0
    for v in values:
        total += v
    context.write(key,total)

os.makedirs(outputDir + '/b/1', exist_ok=True)
jobB1 = Job(inputDir, outputDir + '/b/1', fmapB1, fredB1)
jobB1.setCombiner(fredB1)
success = jobB1.waitForCompletion()

JobB2 calcula la cantidad de páginas distintas visitados por usuario

In [6]:
def fmapB2(key, value, context):
    context.write(key, 1)

def fredB2(key, values, context):
    total = 0
    for v in values:
        total += v
    context.write(key,total)

os.makedirs(outputDir + '/b/2', exist_ok=True)
jobB2 = Job(outputDir + '/b/1', outputDir + '/b/2', fmapB2, fredB2)
jobB2.setCombiner(fredB2)
success = jobB2.waitForCompletion()

JobB3 obtiene el usuario con mas páginas distintas visitadas

In [7]:
def fmapB3(key, value, context):
    context.write(1, (key,value))

def fredB3(key, values, context):
    max = (0,0)
    for v in values:
        if int(max[1]) < int(v[1]):
            max = v
    print(f"El usuario que más páginas distintas visitó es {max[0]} con {max[1]} páginas visitadas")
    context.write(max[0],max[1])

os.makedirs(outputDir + '/b/3', exist_ok=True)
jobB3 = Job(outputDir + '/b/2', outputDir + '/b/3', fmapB3, fredB3)
success = jobB3.waitForCompletion()

El usuario que más páginas distintas visitó es 9458 con 6 páginas visitadas


![b](https://docs.google.com/drawings/d/e/2PACX-1vQrGij1qYqZ70QtamfFUWa_QhQXt5aOE-U3Q--ZIjDeePrBsyNrU3DiFocYwRMKgmyuS2rlhpUctKjN/pub?w=662&h=172)

### c. La página más visitada (en cuanto a cantidad de visitas, sin importar el tiempo de permanencia) por todos los usuarios.

JobC1 calcula cantidad de visitas por página

In [6]:
# <id_user, time, id_page>
def fmapC1(key, value, context):
    values = value.split()
    context.write(values[1], 1)

def fredC1(key, values, context):
    total = 0
    for v in values:
        total += v
    context.write(key,total)

os.makedirs(outputDir + '/c/1', exist_ok=True)
jobC1 = Job(inputDir, outputDir + '/c/1', fmapC1, fredC1)
jobC1.setCombiner(fredC1)
success = jobC1.waitForCompletion()

JobC2 obtiene la página mas visitada

In [9]:
# <id_page, total>
def fmapC2(key, value, context):
    value = value.split()
    context.write(1, (key, value[0]))

def fredC2(key, values, context):
    max = (0,0)
    for v in values:
        if (int(max[1]) < int(v[1]) ):
            max = v
    print(f"La página más visitada es {max[0]} con {max[1]} visitas")
    context.write(max[0],max[1])

os.makedirs(outputDir + '/c/2', exist_ok=True)
jobC2 = Job(outputDir + '/c/1', outputDir + '/c/2', fmapC2, fredC2)
success = jobC2.waitForCompletion()

La página más visitada es 29 con 81 visitas


![JobC](https://docs.google.com/drawings/d/e/2PACX-1vTqOtUNtx9HDNgXOBvSGZ2J2c9KJdB8E-B4SWQyl48iwI2C95krGnM5a2WFLECRUOzLbo4KQLk_TS8v/pub?w=521&h=171)